In [1]:
import requests
import json
import re
from tqdm import tqdm
import os
import pandas as pd
import time
import uuid
from jsonschema import validate

In [2]:
def fetch_save_read(url, file, reader=pd.read_csv, sep='\t', **kwargs):
  ''' Download file from {url}, save it to {file}, and subsequently read it with {reader} using pandas options on {**kwargs}.
  '''
  if not os.path.exists(file):
    if os.path.dirname(file):
      os.makedirs(os.path.dirname(file), exist_ok=True)
    df = reader(url, sep=sep, index_col=None)
    df.to_csv(file, sep=sep, index=False)
  return pd.read_csv(file, sep=sep, **kwargs)

In [3]:
organism = "Mammalia/Homo_sapiens"
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/{}.gene_info.gz'.format(organism)
file = '{}.gene_info.tsv'.format(organism)

ncbi_gene = fetch_save_read(url, file)


In [4]:
ncbi_gene

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410...,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20240411,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899...,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20240416,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000291190|AllianceGe...,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20231010,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171...,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20240403,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156...,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20240403,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193451,741158,8923215,trnD,-,-,-,MT,-,tRNA-Asp,tRNA,-,-,-,-,20200909,-
193452,741158,8923216,trnP,-,-,-,MT,-,tRNA-Pro,tRNA,-,-,-,-,20200909,-
193453,741158,8923217,trnA,-,-,-,MT,-,tRNA-Ala,tRNA,-,-,-,-,20200909,-
193454,741158,8923218,COX1,-,-,-,MT,-,cytochrome c oxidase subunit I,protein-coding,-,-,-,cytochrome c oxidase subunit I,20230818,-


In [5]:
ncbi_gene.type_of_gene.unique()

array(['protein-coding', 'pseudo', 'other', 'unknown', 'ncRNA', 'tRNA',
       'rRNA', 'scRNA', 'snoRNA', 'snRNA', 'biological-region'],
      dtype=object)

In [6]:
def maybe_split(record):
    ''' NCBI Stores Nulls as '-' and lists '|' delimited
    '''
    if record in {'', '-'}:
        return set()
    return set(record.split('|'))

def supplement_dbXref_prefix_omitted(ids):
    ''' NCBI Stores external IDS with Foreign:ID while most datasets just use the ID
    '''
    for id in ids:
        # add original id
        yield id
        # also add id *without* prefix
        if ':' in id:
            yield id.split(':', maxsplit=1)[1]

In [7]:
ncbi_gene['All_synonyms'] = [
    set.union(
      maybe_split(gene_info['Symbol']),
      maybe_split(gene_info['Symbol_from_nomenclature_authority']),
      maybe_split(str(gene_info['GeneID'])),
      maybe_split(gene_info['Synonyms']),
      maybe_split(gene_info['Other_designations']),
      maybe_split(gene_info['LocusTag']),
      set(supplement_dbXref_prefix_omitted(maybe_split(gene_info['dbXrefs']))),
    )
    for _, gene_info in ncbi_gene.iterrows()
  ]

synonyms, gene_id = zip(*{
    (synonym, gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
    for synonym in gene_info['All_synonyms']
  })
ncbi_lookup_syn = pd.Series(gene_id, index=synonyms)
symbols, cap, gene_id = zip(*{
    (gene_info['Symbol'], gene_info['Symbol'].upper(), gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
  })
ncbi_lookup_sym = pd.Series(gene_id, index=symbols)
ncbi_lookup_sym_cap = pd.Series(gene_id, index=cap)

In [8]:
index_values = ncbi_lookup_syn.index.value_counts()
ambiguous = index_values[index_values > 1].index
ncbi_lookup_syn_disambiguated = ncbi_lookup_syn[(
(ncbi_lookup_syn.index == ncbi_lookup_syn) | (~ncbi_lookup_syn.index.isin(ambiguous))
)]
sym_dict = ncbi_lookup_sym.to_dict()
syn_dict_cap = ncbi_lookup_sym_cap.to_dict()
syn_dict = ncbi_lookup_syn_disambiguated.to_dict()
def gene_lookup(gene):
    gene_id = sym_dict.get(gene)
    if gene_id: return str(gene_id)
    gene_id = syn_dict_cap.get(gene)
    if gene_id: return str(gene_id)
    return str(syn_dict.get(gene))

In [9]:
gene_lookup('HLA-A')

'3105'

In [10]:
gene_name_mapper = {
    "KI67": "MKI67",
    "C16ORF61": "CMC2",
    "C20ORF46": "TMEM74B",
    "C9ORF30": "MSANTD3",
    "GUS": "GUSB",
    "RPLPO": "RPLP0",
    "TRFC": "TFRC",
    "TSBP": "TSBP1"
}
invalid_genes = set()
def valid_gene(gene):
    gene = gene_name_mapper.get(gene, gene)
    if gene_lookup(gene) != 'None':
        return True
    invalid_genes.add(gene)
    return False

In [11]:
conditions = {}
invalid_conditions = set()
def get_condition_id(condition):
	if condition in conditions: return conditions
	else:
		res = requests.get("https://www.ebi.ac.uk/ols4/api/v2/entities?search=%s&ontologyId=doid"%condition)
		results = res.json()
		if results['numElements'] > 0:
			for e in results["elements"]:
				# print(e)
				# print(e["label"])
				# print(e["synonym"])
				if type(e["label"]) == str and e["label"].lower() == condition.lower() and 'DOID' in e["curie"] or ("synonym" in e and condition.lower() in [d.lower() for d in e["synonym"] if type(d) == str]):
					print('here')
					conditions[condition] = {
						"label": e["label"],
						"onto_id": e["curie"],
						"source": e["ontologyPreferredPrefix"]
					}
					return conditions
		if condition not in conditions:
			invalid_conditions.add(condition)
			return None
			

		


In [12]:
patents = {
	"WIPO": "https://patentscope.wipo.int/search/en/detail.jsf?docId=%s",
	"Google Patents": "https://patents.google.com/patent/%s/en"
}

In [13]:
with open('data/PMC_biomarkers_051824.gmt', 'w') as fw:
    with open('data/PMC_Gene_Set_Biomarkers-PMC_Biomarker_Sets.tsv') as o:
        for line in o:
            if 'evidence_source' in line:
                continue
            group, c, pubmed, evidence_source, biomarker_temp, best_biomarker_role, *genes = line.split("\t")
            print(best_biomarker_role)
            set_genes = set(genes)
            for item in set_genes.copy():
                if len(item) == 0:
                    set_genes.remove(item)
                if item == '\n':
                    set_genes.remove(item)
            gs_str = '\t'.join(set_genes)
            fw.write(f"{group}\t\t{gs_str}\n")

diagnostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker, diagnostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
diagnostic_biomarker
diagnostic_biomarker, prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
diagnostic_biomarker
monitoring_biomarker
prognostic_biomarker, monitoring_biomarker
diagnostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker, diagnostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognostic_biomarker
prognosti

In [14]:
def list_of_marker_roles(role_string):
    role_array = role_string.split(',')
    for idx, role in enumerate(role_array):
        role_array[idx] = {'role': role.strip().replace("_biomarker", "")}
    return role_array

In [15]:
gmt = []
biomarkers = []
with open('data/PMC_Gene_Set_Biomarkers-PMC_Biomarker_Sets.tsv') as o:
    for line in o:
        if 'evidence_source' in line:
            continue
        group, c, pubmed, evidence_source, biomarker_temp, best_biomarker_role, *genes = line.split("\t")
        print(c)
        conditions = get_condition_id(c)
        if c in conditions:
            biomarker = {
                # "biomarker_id": "GeneSet_%s"%len(biomarkers),
                "best_biomarker_role": list_of_marker_roles(best_biomarker_role),
                "condition": {
                    "id": conditions[c]["onto_id"],
                    "recommended_name": {
                        "id": conditions[c]["onto_id"],
                        "name": conditions[c]["label"],
                        "resource": "Disease Ontology",
                        "url": "http://disease-ontology.org/term/%s"%conditions[c]["onto_id"],
                    }
                },
            }
            evidence_source = []
            evidence_source.append({
                "id": pubmed,
                "database": "PubMed",
                "url": "https://pubmed.ncbi.nlm.nih.gov/%s"%pubmed, 
                "evidence_list":[],
                'tags':[]
            })
            # if patent_id:
            #     evidence_source.append({
            #         "evidence_id": patent_id,
            #         "database": patent_source,
            #         "url": patents[patent_source]%patent_id
            #     })
            biomarker["evidence_source"] = evidence_source
            biomarker_component = []
            for gene in genes:
                evidence_source = []
                evidence_source.append({
                "id": pubmed,
                "database": "PubMed",
                "url": "https://pubmed.ncbi.nlm.nih.gov/%s"%pubmed,
                "evidence_list":[],
                "tags": [{"tag":"assessed_biomarker_entity"}, {"tag":"best_biomarker_role"}]
                })
                gene = gene.strip().upper()
                if valid_gene(gene):
                    gene = gene_name_mapper.get(gene, gene)
                    b = {
                        "biomarker": biomarker_temp.replace("{gene}", gene),
                        "assessed_biomarker_entity": {
                            "recommended_name": gene
                        },
                        "assessed_biomarker_entity_id": "NCBI:"+gene_lookup(gene),
                        "assessed_entity_type": "gene",
                        "evidence_source": evidence_source
                    }
                    biomarker_component.append(b)
            
            biomarker["biomarker_component"] = biomarker_component
            biomarkers.append(biomarker)
# "assessed_entity_type": ncbi_gene[ncbi_gene.GeneID == int(gene_lookup(gene))]["type_of_gene"].to_list()[0] + " gene",



Breast Carcinoma
here
Coronary artery disease
here
Developmental and epileptic encephalopathy 101
here
Rheumatoid Arthritis
here
Colorectal Cancer
here
Iron deficiency anemia
here
Iron deficiency anemia
Iron deficiency anemia
Iron deficiency anemia
Anemia
here
Iron deficiency anemia
Chronic kidney disease
here
Brain glioma
here
Brain glioma
Depressive Disorder
here
Thyroid cancer
here
Chronic Hepatitis B
here
Liver Cirrhosis
here
Hepatocellular Carcinoma
here
Liver Cirrhosis
Hepatocellular Carcinoma
Hepatocellular Carcinoma
Psoriatic Arthritis
here
Heart disease
here
Lung mixed small cell and squamous cell carcinoma
here
Osteosarcoma
here
Head and neck squamous cell carcinoma
here
Chronic Kidney Disease
here
Gastrointestinal system disease
here
Traumatic Brain Injury
here
Subacute delirium
here
Bladder Cancer
here
Postpartum Depression
here
Postpartum Depression
Periodontitis
here
Alzheimer's Disease
here
Traumatic Brain Injury
Malignant pleural mesothelioma
here
Pancreatic Cancer
here

In [16]:
len(biomarkers)

90

In [17]:
schema = requests.get("https://raw.githubusercontent.com/biomarker-ontology/biomarker-partnership/main/schema/v0.3.3/biomarker_schema.json").json()
sample = requests.get("https://raw.githubusercontent.com/biomarker-ontology/biomarker-partnership/main/supplementary_files/sample_data_model_structures/v0.3/sample_biomarker.json").json()

In [18]:
validate(instance=biomarkers, schema=schema)

In [19]:
with open('out/PMC_biomarker_sets.json', 'w') as o:
     json.dump(biomarkers, o)
with open('output/PMC_biomarker_sets.json', 'w') as o:
     json.dump(biomarkers, o)

In [20]:
invalid_conditions

set()

In [21]:
gmt = []
biomarkers = []
with open('data/PMC_Gene_Set_Biomarkers-PMC_Biomarker_Sets.tsv') as o:
    for line in o:
        if 'evidence_source' in line:
            continue
        group, c, pubmed, evidence_source, biomarker_temp, best_biomarker_role, *genes = line.split("\t")
        print(c)
        conditions = get_condition_id(c)
        if c in conditions:
            biomarker = {
                "biomarker_id": "GeneSet_%s"%len(biomarkers),
                "best_biomarker_role": list_of_marker_roles(best_biomarker_role),
                "condition": {
                    "id": conditions[c]["onto_id"],
                    "recommended_name": {
                        "id": conditions[c]["onto_id"],
                        "name": conditions[c]["label"],
                        "resource": "Disease Ontology",
                        "url": "http://disease-ontology.org/term/%s"%conditions[c]["onto_id"],
                    }
                },
            }
            evidence_source = []
            evidence_source.append({
                "id": pubmed,
                "database": "PubMed",
                "url": "https://pubmed.ncbi.nlm.nih.gov/%s"%pubmed, 
                "evidence_list":[],
                'tags':[]
            })
            # if patent_id:
            #     evidence_source.append({
            #         "evidence_id": patent_id,
            #         "database": patent_source,
            #         "url": patents[patent_source]%patent_id
            #     })
            biomarker["evidence_source"] = evidence_source
            biomarker_component = []
            for gene in genes:
                evidence_source = []
                evidence_source.append({
                "id": pubmed,
                "database": "PubMed",
                "url": "https://pubmed.ncbi.nlm.nih.gov/%s"%pubmed,
                "evidence_list":[],
                "tags": [{"tag":"assessed_biomarker_entity"}, {"tag":"best_biomarker_role"}]
                })
                gene = gene.strip().upper()
                if valid_gene(gene):
                    gene = gene_name_mapper.get(gene, gene)
                    b = {
                        "biomarker": biomarker_temp.replace("{gene}", gene),
                        "assessed_biomarker_entity": {
                            "recommended_name": gene
                        },
                        "assessed_biomarker_entity_id": "NCBI:"+gene_lookup(gene),
                        "assessed_entity_type": "gene",
                        "evidence_source": evidence_source
                    }
                    biomarker_component.append(b)
            
            biomarker["biomarker_component"] = biomarker_component
            biomarkers.append(biomarker)
# "assessed_entity_type": ncbi_gene[ncbi_gene.GeneID == int(gene_lookup(gene))]["type_of_gene"].to_list()[0] + " gene",


with open('out/PMC_biomarker_sets_with_test_id.json', 'w') as o:
     json.dump(biomarkers, o)

Breast Carcinoma
Coronary artery disease
Developmental and epileptic encephalopathy 101
Rheumatoid Arthritis
Colorectal Cancer
Iron deficiency anemia
Iron deficiency anemia
Iron deficiency anemia
Iron deficiency anemia
Anemia
Iron deficiency anemia
Chronic kidney disease
Brain glioma
Brain glioma
Depressive Disorder
Thyroid cancer
Chronic Hepatitis B
Liver Cirrhosis
Hepatocellular Carcinoma
Liver Cirrhosis
Hepatocellular Carcinoma
Hepatocellular Carcinoma
Psoriatic Arthritis
Heart disease
Lung mixed small cell and squamous cell carcinoma
Osteosarcoma
Head and neck squamous cell carcinoma
Chronic Kidney Disease
Gastrointestinal system disease
Traumatic Brain Injury
Subacute delirium
Bladder Cancer
Postpartum Depression
Postpartum Depression
Periodontitis
Alzheimer's Disease
Traumatic Brain Injury
Malignant pleural mesothelioma
Pancreatic Cancer
Atrial Septal Defect
Cutaneous Leishmaniasis
Tongue Squamous Cell Carcinoma
Tongue Squamous Cell Carcinoma
COVID-19
Hepatitis B
Drug psychosis
O